In [2]:
import os
import scipy.io.wavfile as wav
import pandas as pd
import numpy as np
import scipy as sp

# Load dataset
dataset = pd.read_csv('/Users/quentinbacquele/Desktop/Internship/Berkeley/Reunion Gray white-eye/data/pitch_contour_dom.csv')

# Parameters
Fs = 44100  # sampling frequency
window_size = 180
fade_length = 500  # length of the fade in and out
volume = 0.1 # range [0.0, 1.0]

output_dir = '/Users/quentinbacquele/Desktop/Internship/Berkeley/Reunion Gray white-eye/pure_sounds4/'
os.makedirs(output_dir, exist_ok=True)

def generate_signal(frequencies, Fs, window_size, fade_length, volume):
    signal = np.array([])
    phase = 0.0  # Initial phase
    up_frequencies = sp.signal.resample(frequencies, len(frequencies) * window_size)
    t = np.arange(len(up_frequencies))
    phase = ((2 * np.pi) * up_frequencies) / Fs
    t_phase = np.cumsum(phase)
    signal = volume * np.sin(t_phase)

    # Apply fade-in
    if fade_length > 0 and len(signal) > fade_length:
        fade_in = np.linspace(0, 1, fade_length)
        signal[:fade_length] *= fade_in

    # Apply fade-out
    if fade_length > 0 and len(signal) > fade_length:
        fade_out = np.linspace(1, 0, fade_length)
        signal[-fade_length:] *= fade_out
    
    return signal

def normalize_waveform(waveform, target_dbfs):
    rms = np.sqrt(np.mean(waveform**2))
    scale = 10**(target_dbfs / 20) / rms
    return np.int16(np.clip(waveform * scale, -1, 1) * 32767)

target_dbfs = -20 

for file_name, group in dataset.groupby('file_name'):
    wav_file_path = os.path.join(output_dir, f"{file_name}")
    frequencies = group['pitch_value'].to_numpy()
    sound_wave = generate_signal(frequencies, Fs, window_size, fade_length, volume)

    sound_wave_normalized = normalize_waveform(sound_wave, target_dbfs)
    wav.write(wav_file_path, Fs, sound_wave_normalized)
